<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-PyTorch/blob/master/Chapter02/Implementing_custom_loss_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.3.3 实现自定义损失函数

在某些情况下, 我们可能必须实现一个针对具体问题的自定义损失函数——特别是对于一些涉及目标检测/生成对抗网络(GAN)的复杂用例.
PyTorch为我们提供了通过编写自己的函数来构建自定义损失函数的功能

在本节中, 我们将实现一个自定义损失函数, 它的功能与nn.Module中预先构建的MSELoss函数相同:

In [1]:
# 1. 导入数据, 构建数据集和DataLoader, 并定义一个神经网络
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

import torch

X = torch.tensor(x).float()
Y = torch.tensor(y).float()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
ds = MyDataset(X, Y)
dl = DataLoader(ds, batch_size=2, shuffle=True)

class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x
mynet = MyNeuralNet().to(device)

C:\Users\86183\AppData\Local\Temp\ipykernel_24676\942007418.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
C:\Users\86183\AppData\Local\Temp\ipykernel_24676\942007418.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [2]:
# 2. 自定义损失函数, 取两个张量对象作为输入, 取它们差的平方, 然后返回平均值:
def my_mean_squared_error(_y, y):
    loss = (_y-y)**2
    loss = loss.mean()
    return loss

In [3]:
# 3. 对于相同的输入和输出对, 在前一节已经使用nn.MSELoss获取均方误差损失
loss_func = nn.MSELoss()
loss_value = loss_func(mynet(X),Y)
print(loss_value)

tensor(121.4717, device='cuda:0', grad_fn=<MseLossBackward0>)


In [4]:
# 4. 同样, 当使用在步骤2中定义的函数时, 损失值的函数如下
my_mean_squared_error(mynet(X),Y)

tensor(121.4717, device='cuda:0', grad_fn=<MeanBackward0>)

In [5]:
# 2.3.4 获取中间层的值
input_to_hidden = mynet.input_to_hidden_layer(X)
hidden_activation = mynet.hidden_layer_activation(input_to_hidden)
